#### Deep Learning

# Overfitting and underfitting
#### Francisco Maiocchi

***
### Introducción

El progreso de los modelos se puede guardar durante y luego del entrenamiento. Esto quiere decir que un modelo puede volver a entrenarse desde donde había frenado sin necesidad de arrancar de nuevo. Esto también permite compartir el modelo para que otras personas puedan recrear el trabajo. Cuando se publican modelos y técnicas en investigación, la mayoría comparte:  

+ El código para crear el modelo.
+ Los pesos o parámetros del modelo.

Compartir esto permite al resto entender como funciona el modelo y les permite probarlos con datos nuevos.

Utiliza KERAS que es la API de alto nivel de TensorFlow. Este ejemplo está en web oficial de TensorFlow en https://www.tensorflow.org/tutorials/keras/save_and_restore_models?hl=es  

Por otro lado, hay muchas formas de guardar modelos de TensorFlow, dependiendo de la API que se utilice.

***
### Imports

In [1]:
# Importo TensorFlow como tf
import tensorflow as tf
# Importo keras
from tensorflow import keras

# Librerias auxiliares
import numpy as np
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function
import os

print(tf.__version__)

1.12.0


***
### Obtención del dataset
Vamos a usar el dataset de MNIST para entrenar el modelo y guardar los pesos. Para acelerar esta demostración, solo vamos a usar los primeros 1000 ejemplos.

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

***
### Definimos el modelo


In [3]:
# Returns a short sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
  
    model.compile(optimizer='adam', 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

    return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


***
### Guardar checkpoints durante el entrenamiento
El uso más común es para guardar checkpoints durante y al final del entrenamiento. De esta forma podemos usar un modelo ya entrenado sin la necesidad de volver a entrenarlo o retomar un entrenamiento donde lo habíamos dejado en caso de que lo hayamos interrumpido.  

tf.keras.callbacks.ModelCheckpoint es un callback que realiza esta tarea. La tarea recibe un par de argumentos de entrada para configurar el checkpoint.  

#### Uso del checkpoint callback

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.2725 - acc: 0.6507
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 2s 2ms/step - loss: 1.1974 - acc: 0.6680 - val_loss: 0.7414 - val_acc: 0.7770
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4269 - acc: 0.8772- ETA: 0s - loss: 0.4356 - acc: 0.
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 643us/step - loss: 0.4302 - acc: 0.8770 - val_loss: 0.5652 - val_acc: 0.8190
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2963 - acc: 0.9260
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 580us/step - loss: 0.2914 - acc

In [6]:
ls {checkpoint_dir}

 El volumen de la unidad C es OS
 El n£mero de serie del volumen es: EC0A-1A5D

 Directorio de C:\Users\Francisco\Desktop\PDS IIA\Deep-Learning\training_1

20/12/2018  17:14    <DIR>          .
20/12/2018  17:14    <DIR>          ..
20/12/2018  17:14                71 checkpoint
20/12/2018  17:14         1.631.508 cp.ckpt.data-00000-of-00001
20/12/2018  17:14               647 cp.ckpt.index
               3 archivos      1.632.226 bytes
               2 dirs  101.139.369.984 bytes libres


Ahora vamos a crear un nuevo modelo sin entrenar. Para cargar un modelo con pesos guardados, se necesita tener un modelo con exactamente la misma arquitectura que el modelo original. Como es exactamente igual, puede compartir los pesos a pesar de ser una instancia diferente.  

Ahora vamos a reconstruir un nuevo modelo y evaluarlo con el set de test (sin entrenarlo previamente).

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 302us/step
Untrained model, accuracy:  8.50%


Ahora cargamos los pesos del checkpoint...

In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 90us/step
Restored model, accuracy: 86.00%


#### Opciones de checkpoint callback
El callback ofrece distintas opciones para dar distintos nombres a los checkpoints y ajustar la frecuencia.  

Vamos a entrenar un nuevo modelo y guardar checkpoints con nombres únicos cada 5 épocas.

In [9]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 000

In [10]:
ls {checkpoint_dir}

 El volumen de la unidad C es OS
 El n£mero de serie del volumen es: EC0A-1A5D

 Directorio de C:\Users\Francisco\Desktop\PDS IIA\Deep-Learning\training_2

20/12/2018  17:23    <DIR>          .
20/12/2018  17:23    <DIR>          ..
20/12/2018  17:23                81 checkpoint
20/12/2018  17:22         1.631.508 cp-0005.ckpt.data-00000-of-00001
20/12/2018  17:22               647 cp-0005.ckpt.index
20/12/2018  17:22         1.631.508 cp-0010.ckpt.data-00000-of-00001
20/12/2018  17:22               647 cp-0010.ckpt.index
20/12/2018  17:22         1.631.508 cp-0015.ckpt.data-00000-of-00001
20/12/2018  17:22               647 cp-0015.ckpt.index
20/12/2018  17:22         1.631.508 cp-0020.ckpt.data-00000-of-00001
20/12/2018  17:22               647 cp-0020.ckpt.index
20/12/2018  17:22         1.631.508 cp-0025.ckpt.data-00000-of-00001
20/12/2018  17:22               647 cp-0025.ckpt.index
20/12/2018  17:22         1.631.508 cp-0030.ckpt.data-00000-of-00001
20/12/2018  17:22              

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

Para testear vamos a resetear el modelo y cargarlo con el último checkpoint.

In [12]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 224us/step
Restored model, accuracy: 87.60%


***
### Que son estos archivos?
Los códigos de arriba guardan los pesos en una colección de archivos con formato checkpoint que contienen solo los pesos en formato binario.

***
### Guardar los pesos manualmente
Arriba vimos como cargar los pesos en un modelo.  

Guardar los pesos manualmente es igual de simple, solo se debe usar Model.save_weights.

In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 232us/step
Restored model, accuracy: 87.60%


***
### Guardar el modelo entero
Se puede guardar el modelo entero en un archivo que contiene los pesos, la configuración del modelo y hasta la configurazion del optimizer. Esto permite guardar un checkpoint y continuar el entrenamiento más adelante.  

Guardar un modelo completamente funcional es muy útil. Se puede cargar en TensorFlow.js (HDF5, saved model) y luego entrenarlo y correrlo en un navegador o convertirlo en una aplicación movil con TensorFlow Lite (HDF5, Saved Model).

#### Como un archivo HDF5
Keras provee un formato de archivo básico para guardar llamado HDF5. Para nuestro propósito se puede considerar un archivo binario.

In [14]:
model = create_model()

# You need to use a keras.optimizer to restore the optimizer state from an HDF5 file.
model.compile(optimizer='adam', 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1355 - acc: 0.6780
Epoch 2/5
1000/1000 [==============================] - 0s 409us/step - loss: 0.4173 - acc: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 386us/step - loss: 0.2812 - acc: 0.9140
Epoch 4/5
1000/1000 [==============================] - 0s 396us/step - loss: 0.2025 - acc: 0.9520 0s - loss: 0.2064 - acc: 0.953
Epoch 5/5
1000/1000 [==============================] - 0s 401us/step - loss: 0.1550 - acc: 0.9640


Ahora recreamos el modelo desde ese archivo.

In [15]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Checkeamos su efectividad.

In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 264us/step
Restored model, accuracy: 85.20%


Esta técnica guarda todo:  

+ Los pesos.
+ La configuración del modelo (arquitectura).
+ La configuración del optimizer.

Keras guarda los modelo inspeccionando la arquitectura. Actualmente no es capaz de guardar los optimizers desde tf.train. Cuando se usa uno de estos es necesario recompilar el modelo luego de cargarlo y se pierde el estado del optimizer.  